In [2]:
import os 
import json
import numpy as np
import pandas as pd
import dill as pickle
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import BayesianRidge, LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [26]:
data = pd.read_csv('./data/WarrantyData.csv')
list(data)
data = data[['DeviceSpeed',
 'DynamicAcceleration',
 'EnvTemperature',
 'HeatGenerated',
 'Humidity',
 'ImpactAcceleration',
 'LateralAcceleration',
 'Latitude',
 'Longitude', 
 'RepairedNo',
 'Strain',
 'StructureDistortion',
 'StructureMovement',
 'TransverseDeformation',
 'UsagePerDay',
 'VerticalContactForce',
 'VerticalDisplacement',
 'Vibration:',
 'Voltage',
 'WindSpeed',
 'DateTime',
 'DeviceCategory', 
 'Noofdevices']]
data['DateTime'] = pd.to_datetime(data['DateTime'], format='%Y-%m-%d')

data = data.copy()
data['Year'] = data['DateTime'].dt.year
data['Month'] = data['DateTime'].dt.month
data['Day'] = data['DateTime'].dt.day

del data['DateTime']

encoder = preprocessing.LabelEncoder()
encoder.fit(data['DeviceCategory'])
data['DeviceCategory'] = encoder.transform(data['DeviceCategory'])

data.head()

,DeviceSpeed,DynamicAcceleration,EnvTemperature,HeatGenerated,Humidity,ImpactAcceleration,LateralAcceleration,Latitude,Longitude,RepairedNo,...,VerticalContactForce,VerticalDisplacement,Vibration:,Voltage,WindSpeed,DeviceCategory,Noofdevices,Year,Month,Day
0,1188,11,42,94,54,6,200,19,74,2,...,1654,39,130,177,16,1,42,2016,12,8
1,1163,16,35,52,99,12,94,33,16,4,...,1859,43,136,270,17,2,29,2016,12,8
2,1083,222,34,60,74,28,6,83,84,5,...,1686,42,20,185,11,4,33,2016,12,8
3,1607,189,23,72,68,5,50,75,121,2,...,1765,27,162,224,5,0,23,2016,12,8
4,1334,15,24,51,35,6,135,31,81,0,...,1676,32,76,233,36,3,0,2016,12,8


In [27]:
#Split Data
msk = np.random.rand(len(data)) < 0.8
traindata = data[msk]
testdata= data[~msk]
print('Traning Data => ',len(traindata))
print('Test Data => ',len(testdata))

Traning Data =>  2195
Test Data =>  545


In [43]:
feature_names = ['DeviceCategory','Year','Month','Day']
X_train = traindata[feature_names]
y_train = traindata['Noofdevices']

X_test = testdata[feature_names]
y_test = testdata['Noofdevices']
testdata.head()

,DeviceSpeed,DynamicAcceleration,EnvTemperature,HeatGenerated,Humidity,ImpactAcceleration,LateralAcceleration,Latitude,Longitude,RepairedNo,...,VerticalContactForce,VerticalDisplacement,Vibration:,Voltage,WindSpeed,DeviceCategory,Noofdevices,Year,Month,Day
3,1607,189,23,72,68,5,50,75,121,2,...,1765,27,162,224,5,0,23,2016,12,8
8,846,176,43,55,31,17,245,74,154,3,...,1313,41,201,239,34,0,42,2016,12,9
11,1346,175,46,40,65,0,79,85,126,0,...,1563,48,138,260,34,2,52,2016,12,10
15,909,139,31,78,87,29,222,49,147,1,...,1589,4,100,249,4,1,36,2016,12,11
29,576,146,54,76,96,25,88,27,136,2,...,1785,10,61,193,46,3,7,2016,12,13


In [35]:
#apply scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [46]:
#Linear Regression
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'.format(linreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'.format(linreg.score(X_test, y_test)), end="\n\n")

#Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)), end="\n\n")

#Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)), end="\n\n")
#Setting max decision tree depth to help avoid overfitting
clf2 = DecisionTreeClassifier(max_depth=3).fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf2.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(clf2.score(X_test, y_test)), end="\n\n")     

#Apply K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}' . format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}' . format(knn.score(X_test, y_test)), end="\n\n")

#Linear Discriminant Analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Accuracy of LDA classifier on training set: {:.2f}'.format(lda.score(X_train, y_train)))
print('Accuracy of LDA classifier on test set: {:.2f}'.format(lda.score(X_test, y_test)), end="\n\n")

#Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print('Accuracy of GNB classifier on training set: {:.2f}'.format(gnb.score(X_train, y_train)))
print('Accuracy of GNB classifier on test set: {:.2f}'.format(gnb.score(X_test, y_test)), end="\n\n")

#Support Vector Machine
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'.format(svm.score(X_train, y_train)))
print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(X_test, y_test)), end="\n\n")




filename = './model/model_warranty_estimation.pk'
with open(filename, 'wb') as file:
    pickle.dump(clf, file)

Accuracy of Logistic regression classifier on training set: 0.00
Accuracy of Logistic regression classifier on test set: 0.01

Accuracy of Logistic regression classifier on training set: 0.03
Accuracy of Logistic regression classifier on test set: 0.03

Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of Decision Tree classifier on test set: 0.02

Accuracy of Decision Tree classifier on training set: 0.04
Accuracy of Decision Tree classifier on test set: 0.02

Accuracy of K-NN classifier on training set: 0.23
Accuracy of K-NN classifier on test set: 0.02

Accuracy of LDA classifier on training set: 0.03
Accuracy of LDA classifier on test set: 0.03

Accuracy of GNB classifier on training set: 0.03
Accuracy of GNB classifier on test set: 0.01

Accuracy of SVM classifier on training set: 0.28
Accuracy of SVM classifier on test set: 0.01

